In [13]:
import xmltodict, json, feedparser

In [14]:
rss_info = feedparser.parse('http://dart.fss.or.kr/api/todayRSS.xml')
rss_info

{'bozo': False,
 'entries': [],
 'feed': {'title': 'DART : 최근공시',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://dart.fss.or.kr/api/todayRSS.xml',
   'value': 'DART : 최근공시'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'http://dart.fss.or.kr'}],
  'link': 'http://dart.fss.or.kr',
  'subtitle': 'DART 전자공시 시스템 - 최근 공시 정보를 제공합니다.',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://dart.fss.or.kr/api/todayRSS.xml',
   'value': 'DART 전자공시 시스템 - 최근 공시 정보를 제공합니다.'},
  'language': 'ko',
  'rights': 'DART',
  'rights_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://dart.fss.or.kr/api/todayRSS.xml',
   'value': 'DART'},
  'published': 'Mon, 22 Aug 2022 21:35:43 GMT',
  'published_parsed': time.struct_time(tm_year=2022, tm_mon=8, tm_mday=22, tm_hour=21, tm_min=35, tm_sec=43, tm_wday=0, tm_yday=234, tm_isdst=0),
  'authors': [{'name': 'DART'}],
  'author': 'DART',
  'author_detail

In [11]:
with open('http://dart.fss.or.kr/api/todayRSS.xml') as dx:
  doc = xmltodict.parse(dx.read())

doc

OSError: [Errno 22] Invalid argument: 'https://dart.fss.or.kr/api/todayRSS.xml'

In [ ]:
print('hello again')

hello again
